In [1]:
import random
from datasketch import MinHash, MinHashLSH
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
articles = [] 

In [18]:
from summarization import Summarizer
from page_news import PageNews
import json
page_name = "Donald Trump"
summarizer = Summarizer(False)
page_news_obj = PageNews(summarizer, page_name)
json_pages = page_news_obj.fetch_news_article(20)

json_pages_parsed = json.loads(json_pages)
for json_page in json_pages_parsed:
    articles.append(json_page["title"] + json_page["content"])

ERROR:root:Failed to fetch data from https://newsapi.org/v2/everything?q=Donald Trump&from=2024-10-14&sortBy=publishedAt&apiKey=f049190b6b744976b46acc21a25972f9: 426 Client Error: Upgrade Required for url: https://newsapi.org/v2/everything?q=Donald%20Trump&from=2024-10-14&sortBy=publishedAt&apiKey=f049190b6b744976b46acc21a25972f9


Page accessed successfully.
Page accessed successfully.
Page accessed successfully.
Page accessed successfully.
Page accessed successfully.
Page accessed successfully.
Fetched data from APIs...
[{'url': 'https://www.gurufocus.com/news/2616598/microsoft-stake-slashed-lakehouse-capital-sells-5000-shares-cuts-value-to-4-million', 'title': '\n          Microsoft Stake Slashed: Lakehouse Capital Sells 5,000 Shares, Cuts Value to $4 Million\n        ', 'content': "As disclosed in its most recent regulatory filing, Lakehouse Capital Pty Ltd has significantly reduced its stake in Microsoft Corporation (MSFT, Financial). The firm trimmed its holdings by 34.5%, bringing the total down to 9,472 shares from over 14,000 previously. The current value of these shares stands at approximately $4.08 million. This move reduces Microsoft's weight in the fund's portfolio to 2.9%, making it the 13th largest holding.\nThe reduction aligns with trends adopted by most institutional investors. For example, Chri

In [19]:
len(articles)

6

In [20]:
# Parameters for LSH
shingle_size = 5  # Number of words in each shingle
num_perm = 128  # Number of permutations for MinHash
threshold = 0.5  # Similarity threshold for LSH

In [21]:
def get_shingles(text, shingle_size):
    words = text.split()
    return set([' '.join(words[i:i + shingle_size]) for i in range(len(words) - shingle_size + 1)])

In [22]:
# Initialize MinHashLSH
lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)

# Store original article and its MinHash
minhash_objects = []

In [23]:


# Process each article
for idx, article in enumerate(articles):
    # Create shingles
    shingles = get_shingles(article, shingle_size)

    # Create MinHash
    m = MinHash(num_perm=num_perm)
    for shingle in shingles:
        m.update(shingle.encode('utf8'))

    # Store MinHash object
    minhash_objects.append((article, m))

    # Add to LSH
    if f"article_{idx}" not in lsh.keys:
        lsh.insert(f"article_{idx}", m)
    else:
        print(f"Key article_{idx} already exists. Skipping insertion.")



In [ ]:
buckets = {}
for idx, (article, m) in enumerate(minhash_objects):
    bucket_ids = lsh.query(m)
    print("similar articles to article", idx)
    print("are in bucket", "bucket_ids)
    if bucket_ids:
        primary_bucket = bucket_ids[0]  # Use the first bucket ID
        if primary_bucket not in buckets:
            buckets[primary_bucket] = article


In [27]:
# Output one representative article from each bucket
print(f"Number of unique buckets: {len(buckets)}\n")
for bucket_id, representative_article in buckets.items():
    print(f"Bucket {bucket_id}: {representative_article}")  # Print first 100 characters



Number of unique buckets: 3

Bucket article_0: 
          Microsoft Stake Slashed: Lakehouse Capital Sells 5,000 Shares, Cuts Value to $4 Million
        As disclosed in its most recent regulatory filing, Lakehouse Capital Pty Ltd has significantly reduced its stake in Microsoft Corporation (MSFT, Financial). The firm trimmed its holdings by 34.5%, bringing the total down to 9,472 shares from over 14,000 previously. The current value of these shares stands at approximately $4.08 million. This move reduces Microsoft's weight in the fund's portfolio to 2.9%, making it the 13th largest holding.
The reduction aligns with trends adopted by most institutional investors. For example, Christopher J. Hasenberg Inc. bought 169.2% more Microsoft this year, and Bellevue Group AG bought 47.5%. At the moment, the combined stake of institutional investors and hedge funds is 71.13 % of Microsoft's total stake.
Microsoft still holds a key position in the technological sector, and its stock has increase